# Quantum Fourier Transform (QFT) Implementation
The Quantum Fourier Transform (QFT) is a quantum version of the discrete Fourier transform (DFT). It transforms a quantum state into a superposition of states, each representing a Fourier coefficient.

The QFT is a key building block in many quantum algorithms, such as Shor's algorithm for factoring. In this notebook, we will:
1. Implement the QFT on a quantum circuit
2. Understand the role of each gate
3. Test the QFT with an example quantum state


In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

## Step 1: Import Necessary Libraries
- Qiskit:
  - QuantumCircuit for building the quantum circuit
  - transpile to optimize the circuit for the simulator
- AerSimulator as a backend for running and testing circuits
- NumPy to handle mathematical operations like pi


In [2]:
def apply_qft(qc, n):
    for i in range(n):
        qc.h(i)
        for j in range(i + 1, n):
            qc.cp(np.pi / (2 ** (j - i)), j, i)
    for i in range(n // 2):
        qc.swap(i, n - i - 1)


## Step 2: Define the QFT Function
The apply_qft function applies the QFT to the first n qubits of a quantum circuit:
1. Hadamard Gate:
   - Applied to each qubit to create a superposition
2. Controlled Phase Rotations:
   - Introduce relative phases between states, encoding the Fourier transform
   - Angle for each gate is pi divided by 2 raised to the power of the qubit distance
3. Swap Qubits:
   - Reverses the order of qubits to match the standard QFT output


In [3]:
def apply_inverse_qft(qc, n):
    for i in range(n // 2):
        qc.swap(i, n - i - 1)
    for i in range(n - 1, -1, -1):
        for j in range(n - 1, i, -1):
            qc.cp(-np.pi / (2 ** (j - i)), j, i)
        qc.h(i)


## Step 3: Define the Inverse QFT Function
The apply_inverse_qft function applies the inverse Quantum Fourier Transform (QFT†):
1. Reverse Qubits:
   - Swaps the qubits back to their original order
2. Undo Phase Rotations:
   - Controlled phase gates are reversed with a negative angle
3. Undo Hadamard Gates:
   - Reverts qubits to their original state


In [4]:
def test_qft(input_state):
    n = len(input_state)
    qc = QuantumCircuit(n)
    for i, bit in enumerate(reversed(input_state)):
        if bit == 1:
            qc.x(i)
    apply_qft(qc, n)
    qc.measure_all()
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit)
    result = job.result()
    return result.get_counts()

input_state = [1, 0, 1]
qft_result = test_qft(input_state)
print(qft_result)


{'100': 124, '001': 141, '101': 134, '111': 123, '011': 125, '010': 118, '000': 129, '110': 130}


## Step 4: Test the QFT Function
The test_qft function:
1. Initializes the quantum circuit to a binary input state
   - Example: input [1, 0, 1] corresponds to the quantum state |101>
2. Applies the QFT on the quantum state
3. Simulates the circuit and retrieves the results
